In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
from keras.datasets import mnist

import keras
from defense import *
import tensorflow as tf
from torch.autograd import Variable
import torch
import numpy as np
import time
#from wrapper import Model

/home/yusu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Uppercase local vars:
	BATCH_SIZE: 64
	CRITIC_ITERS: 5
	DATA_DIR: cifar-10-batches-py
	DIM: 64
	ITERS: 200000
	LAMBDA: 10
	MODE: wgan-gp
	OUTPUT_DIM_CIFAR10: 3072
	OUTPUT_DIM_MNIST: 784


In [2]:
class blackbox:
    def __init__(self,model):
        self.model = model
        
    def attack_untargeted(self, x0, y0, shape ,alpha = 2, beta = 0.005, iterations = 1000):
        """ Attack the original image and return adversarial example
            model: (pytorch model)
            alpha: learning rate 
            beta: learning rate
            train_dataset: set of training data
            (x0, y0): original image
        """

        
        if (self.model.predict(x0) != y0):
            print("Fail to classify the image. No need to attack.")
            return torch.zeros(shape),0
        
        num_directions = 50
        best_theta, g_theta = None, float('inf')
        query_count = 0
        print("original label is ", y0)
        
        timestart = time.time()
        for i in range(num_directions):
            #print("generating a new distortion")
            theta = torch.randn(shape)*10
            #print(theta.size())
            initial_lbd = torch.norm(theta)
            theta = theta/initial_lbd
            pred,_ = self.model.predict_gan(theta*initial_lbd,x0)
            #print("predicted label is", pred)
            
            if pred != y0:
                print("new feasible direction and iteration", pred,i)
                lbd, count = self.fine_grained_binary_search( x0, y0, theta, initial_lbd, g_theta)
                query_count += count
                if lbd < g_theta:
                    best_theta, g_theta = theta,lbd
                    print("--------> Found distortion %.4f" % g_theta)
        timeend = time.time()
        print("==========> Found best distortion %.4f in %.4f seconds using %d queries" % (g_theta, timeend-timestart, query_count))
    

        g1 = 1.0
        theta, g2 = best_theta.clone(), g_theta
        torch.manual_seed(0)
        opt_count = 0
        stopping = 0.01
        prev_obj = 100000
        time1 = time.time()
        new_lb,orig_mod = self.model.predict_gan(best_theta*g_theta,x0)
        time2 = time.time()
        print("new label is :", new_lb)
        print("time consuming for one query -- gan:", time2-time1)
        for i in range(iterations):
            print("this is interation:",i)
            _,orig_mod = self.model.predict_gan(best_theta*g_theta,x0)
            print("loc1")
            mod_norm = np.linalg.norm(orig_mod)
            if mod_norm < 8:
                print("====================query number after distortion < 1 =======================: ",opt_count)
                break
            
            gradient = torch.zeros(theta.size())
            q = 5
            min_g1 = float('inf')
            for _ in range(q):
                u = torch.randn(theta.size()).type(torch.FloatTensor)
                u = u/torch.norm(u)
                ttt = theta+beta * u
                ttt = ttt/torch.norm(ttt)
                g1, count = self.fine_grained_binary_search_local(x0, y0, ttt, initial_lbd = g2, tol=beta/500)
                opt_count += count
                gradient += (g1-g2)/beta * u
                if g1 < min_g1:
                    min_g1 = g1
                    min_ttt = ttt
            gradient = 1.0/q * gradient
            print("loc2")
            
    
            if (i+1)%50 == 0:
                print("Iteration %3d: g(theta + beta*u) = %.4f g(theta) = %.4f distortion %.4f num_queries %d" % (i+1, g1, g2, torch.norm(g2*theta), opt_count))
                if g2 > prev_obj-stopping:
                    break
                prev_obj = g2
    
            min_theta = theta
            min_g2 = g2
        
            for _ in range(15):
                new_theta = theta - alpha * gradient
                new_theta = new_theta/torch.norm(new_theta)
                new_g2, count = self.fine_grained_binary_search_local( x0, y0, new_theta, initial_lbd = min_g2, tol=beta/500)
                opt_count += count
                alpha = alpha * 2
                if new_g2 < min_g2:
                    min_theta = new_theta 
                    min_g2 = new_g2
                else:
                    break
            print("loc3")
    
            if min_g2 >= g2:
                for _ in range(15):
                    alpha = alpha * 0.25
                    new_theta = theta - alpha * gradient
                    new_theta = new_theta/torch.norm(new_theta)
                    new_g2, count = self.fine_grained_binary_search_local( x0, y0, new_theta, initial_lbd = min_g2, tol=beta/500)
                    opt_count += count
                    if new_g2 < g2:
                        min_theta = new_theta 
                        min_g2 = new_g2
                        break
            print("loc4")
    
            if min_g2 <= min_g1:
                theta, g2 = min_theta, min_g2
            else:
                theta, g2 = min_ttt, min_g1
    
            if g2 < g_theta:
                best_theta, g_theta = theta.clone(), g2
            
            if alpha < 1e-4:
                alpha = 1.0
                print("Warning: not moving, g2 %lf gtheta %lf" % (g2, g_theta))
                beta = beta * 0.1
                if (beta < 0.00005):

                    break
    
        print("defensegan")
        print("best distortion :", g_theta)
        print("number of queries :", opt_count+query_count)
        mod_gan = np.array(g_theta*best_theta)
        print("return g_theta*best_theta, shape of it:", mod_gan.shape)
        return mod_gan, opt_count+query_count
    def fine_grained_binary_search_local(self, x0, y0, theta, initial_lbd = 1.0, tol=1e-3):
        nquery = 0
        theta = np.array(theta)
        initial_lbd = np.array(initial_lbd)
        lbd = initial_lbd
        pred,mod = self.model.predict_gan(lbd*theta,x0)
        print("predicted label in local binary search:", pred)
        if pred == y0:
            lbd_lo = lbd
            lbd_hi = lbd*1.01
            nquery += 1
            #pred1,_ = self.model.predict_gan(lbd_hi*theta,x0)
            while self.model.predict_gan(lbd_hi*theta,x0)[0] == y0:
                lbd_hi = lbd_hi*1.01
                nquery += 1
                if lbd_hi > 100:
                    return float('inf'), nquery

        else:
            lbd_hi = lbd
            lbd_lo = lbd*0.99
            nquery += 1
            while self.model.predict_gan(lbd_lo*theta,x0)[0] != y0 :
                lbd_lo = lbd_lo*0.99
                nquery += 1
        timestart1 = time.time()   
        while not np.isclose(lbd_hi,lbd_lo,tol):
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.model.predict_gan(lbd_mid*theta,x0)[0] != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
            #print("countinue while, hi and lo:", lbd_hi,lbd_lo)
        #timeend1 = time.time()
        #print("time consuming in 3rd while loop")
        return lbd_hi, nquery
        

    def fine_grained_binary_search(self, x0, y0,theta, initial_lbd, current_best):
        nquery = 0
        theta = np.array(theta)
        initial_lbd = np.array(initial_lbd)
        if initial_lbd > current_best:
            pred,_ = self.model.predict_gan(current_best*theta,x0)
            if pred == y0:
                nquery += 1
                return float('inf'), nquery
            lbd = current_best
        else:
            lbd = initial_lbd
        
        lbd_hi = lbd
        lbd_lo = 0
    
        while not np.isclose(lbd_hi,lbd_lo,1e-5):
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
#            modi = self.get_modifier(lbd_mid*theta,x0,gan)
#            print("type of modi:", type(modi))
            #pred,_ = self.model.predict_gan(lbd_mid*theta,x0)
            if self.model.predict_gan(lbd_mid*theta,x0)[0] != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
        return lbd_hi, nquery
    


In [3]:
class Model:
    def __init__(self,model,bounds,sess,gan):
        self.model = model
        self.bounds = bounds
        self.gan = gan
        self.sess = sess
    
    def predict_gan(self,mod,x0):
        orig_mod = np.expand_dims(np.array(mod),0)
        x_new = tf.placeholder(tf.float32,(1,128))
        mod = self.gan(x_new)
        mod = self.sess.run(mod,{x_new:orig_mod})
        mod = np.sum(mod,0)
        pred = self.predict(x0+mod)
        return pred,mod

        
    def predict(self,image):
        #print("shape of image:", image.shape)
        #image = tf.convert_to_tensor(image,np.float32)
        if isinstance(image, np.ndarray ):
            if self.bounds[1] == 255:
                new_img = image*255
                new_img = np.clip(new_img,0,255)
            else:
                new_img = np.clip(image,0,1)
            new_img= np.expand_dims(new_img,0)
        else:     
            if self.bounds[1] == 255:
                new_img = image*255
                new_img = tf.clip_by_value(new_img,0,255)
                new_img = tf.expand_dims(new_img,0)
            else:
                new_img = tf.clip_by_value(image,0,1)
                new_img = tf.expand_dims(new_img,0)
            with self.sess.as_default():
                new_img.eval()
        
        
        #print("type of image after clip:",type(new_img))

        labels = self.model.predict(new_img)
        #print(labels)
        label = np.argmax(labels[0])
        #print("the current label: ", label)
        return label
    

In [4]:
session = keras.backend.get_session()

In [4]:
keras.backend.set_learning_phase(False)
model = keras.models.load_model("data/mnist")
model = Model(model,[0.0,1.0],session,lambda x : Generator(1,x))

#touse = [x for x in tf.trainable_variables() if 'Generator' in x.name]
#saver = tf.train.Saver(touse)
#saver.restore(session, 'data/mnist-gan')

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_test = np.array(x_test, dtype=np.float32)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_test /= 255.0
image = x_test[:2]

xin = tf.placeholder(tf.float32, [1, 128])
mygan = Generator(1, xin)
theta,g_theta = None,float('inf')
flag = False
for i in range(1000):
    theta = torch.randn((1,128)).type(torch.FloatTensor)
    #theta = np.random.randn(1,128)
    g_theta = torch.norm(theta)
    theta = theta/g_theta
    g_theta = np.array(g_theta)*8
    theta = np.array(theta)
    new_mod = session.run(mygan,{xin:theta*g_theta})
    mod = np.sum(new_mod,0)
    new_pred = model.predict(image[0]+mod)
    if  new_pred != y_test[0]:
        print("new_pred:",new_pred)
        print("new_distortion",torch.norm(mod))
        print("find an availabel theta")
        flag = True
        break
        
if flag:
    print("True label", y_test[0])
    print("Preds",model.predict(image[0]))    
    for i in range(20):
        new_mod = session.run(mygan,{xin:theta*g_theta})
        mod = np.sum(new_mod,0)
        distortion = np.linalg.norm(mod)
        pred = model.predict(image[0]+mod)
        print("predicted label is %3d, distortion is %.4f"%(pred,distortion))
        g_theta *= 0.9

if flag == False:
    print("did not find an availabel initialization")

NameError: name 'keras' is not defined

In [ ]:
keras.backend.set_learning_phase(False)
model = keras.models.load_model("data/mnist")

model = Model(model,[0.0,1.0],session,lambda x : Generator(1,x))


(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_test = np.array(x_test, dtype=np.float32)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_test /= 255.0
image = x_test[:20]
shape = 128
xin = tf.placeholder(tf.float32, [1, 128])
mygan = Generator(1, xin)

attack = blackbox(model)

dist = []
count = []
for i in range(10):
    print("=====================attacking image %2d =========================="%(i+2))
    print("label of pure image:", model.predict(image[i]))
    adv_mod,query = attack.attack_untargeted(image[i],y_test[i],shape,alpha = 0.2, beta = 0.05, iterations = 1000)
    adv_mod = np.expand_dims(np.array(adv_mod),0)
    mod = session.run(mygan,{xin:adv_mod})
    adv = np.sum(mod,0)+image[i]
    print("new label of adv_sample :", model.predict(adv))
    dist.append(np.linalg.norm(mod))
    count.append(query)

index = np.nonzero(count)
index = list(index)[0].tolist()

avg_distortion = np.mean(np.array(dist)[index])
avg_count = np.mean(np.array(count)[index])
print("the average distortion for %2d images :"%(len(index)),avg_distortion)
for i in dist:
    print(i)
print("the number of queries for %2d images :"%(len(index)), avg_count)
for j in count:
    print(j)


=====================attacking image  2 ==========================
label of pure image: 7
original label is  7
new feasible direction and iteration 8 0
--------> Found distortion 34.1475
new feasible direction and iteration 2 1
new feasible direction and iteration 8 2
new feasible direction and iteration 2 3
new feasible direction and iteration 2 4
new feasible direction and iteration 2 5
new feasible direction and iteration 8 6
new feasible direction and iteration 8 7
new feasible direction and iteration 8 8
new feasible direction and iteration 8 9
--------> Found distortion 27.6737
new feasible direction and iteration 8 10
new feasible direction and iteration 8 11
new feasible direction and iteration 8 12
new feasible direction and iteration 8 13
new feasible direction and iteration 8 14
new feasible direction and iteration 8 15
new feasible direction and iteration 8 16
new feasible direction and iteration 8 17
--------> Found distortion 22.2628
new feasible direction and iteration 8

predicted label in local binary search: 8
after 1st binary search
before 1st binary search
predicted label in local binary search: 7
after 1st binary search
before 1st binary search
predicted label in local binary search: 8
after 1st binary search
before 1st binary search
predicted label in local binary search: 7
after 1st binary search
before 1st binary search
predicted label in local binary search: 7
after 1st binary search
loc2
predicted label in local binary search: 7
loc3
predicted label in local binary search: 7
predicted label in local binary search: 8
loc4
this is interation: 6
loc1
before 1st binary search
predicted label in local binary search: 7
after 1st binary search
before 1st binary search
predicted label in local binary search: 8
after 1st binary search
before 1st binary search
predicted label in local binary search: 7
after 1st binary search
before 1st binary search
predicted label in local binary search: 7
after 1st binary search
before 1st binary search
predicted lab